# Plotting locations in Nordisk Familjebok

In [1]:
import sys
sys.path.append('../../')  # Assuming the parent directory

import matplotlib.pyplot as plt
# from mpl_toolkits.basemap import Basemap
#import scripts.coordinates_retreival as gs
# import json
# import time
from utils import json_helpers as jh
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd

e1 = "e1_linked2"

### Functions helpful for comparision of editions

In [ ]:
# Returns all coordinates to articles in an edition
def get_all_coords(edition: list[dict]):
    return [(entry.get('latitude', None), (entry.get('longitude', None))) for entry in edition]

# Returns the location entries in both editions
def coords_union(edition1: list[dict], edition2: list[dict]) -> list[dict]:
    coords1 = get_all_coords(edition1)
    coords2 = get_all_coords(edition2)
    
    union_coords = set(coords1 + (coords2))
    union_coords.discard((None, None))
    
    qids = set()
    entry_union = []
    for entry in edition1 + edition2:
        lat = entry.get('latitude', None)
        lon = entry.get('longitude', None)
        qid = entry['qid']
        if (lat, lon) in union_coords and qid not in qids:
            entry_union.append(entry)

    return entry_union

# Returns the location entries in edition1 but not in edition2
def coords_diff(edition1: dict, edition2: dict) -> dict:
    coords1 = set(get_all_coords(edition1))
    coords2 = set(get_all_coords(edition2))

    diff_coords = coords1.difference(coords2)
    diff_coords.discard((None, None))

    qids = set()
    entry_diff = []
    for entry in edition1:
        lat = entry.get('latitude', None)
        lon = entry.get('longitude', None)
        qid = entry['qid']
        if (lat, lon) in diff_coords and qid not in qids:
            diff_coords.discard((lat, lon))
            entry_diff.append(entry)

    return entry_diff

def coords_intersec(edition1: dict, edition2: dict) -> dict:
    coords1 = set(get_all_coords(edition1))
    coords2 = set(get_all_coords(edition2))

    intersec_coords = coords1.intersection(coords2)
    intersec_coords.discard((None, None))


    qids = set()
    entry_intersec = []
    for entry in edition1 + edition2:
        lat = entry.get('latitude', None)
        lon = entry.get('longitude', None)
        qid = entry['qid']
        if (lat, lon) in intersec_coords and qid not in qids:
            qids.add(entry['qid'])
            entry_intersec.append(entry)

    return entry_intersec

### Visualization functions

In [ ]:
def twod_map_coords(entries: list[dict], samples = 20):
    # output_image = config["visualization"]["output_image"]
    data = pd.DataFrame(entries)[:samples]

    fig = px.scatter_geo(data, lat="latitude", 
                         lon="longitude", 
                         hover_name='qid',
                         hover_data='text',
                         projection="natural earth", opacity=0.3
                         )
    # fig.to_image(output_image)

    fig.update_layout(hoverlabel=dict(
        bgcolor="white",
        font_size=6,
        font_family="Rockwell",
    ))

    fig.write_html("2d_plot.html")
    fig.show()



def threed_map_coords(config):
    FILENAME_OUT_CSV = config["coords_fetch"]["output_csv_file"]

    lons, lats = [], []

    with open(FILENAME_OUT_CSV) as f:
        for line in f.readlines():
            lon, lat = line.split(',')

            # print(lon, lat)
            lons.append(float(lon))
            lats.append(float(lat))

    print(f"Successful coords: {len(lons)}")

    # if you are passing just one lat and lon, put it within "[]"
    # editing the marker
    fig = go.Figure(go.Scattergeo(lat=lats, lon=lons))
    # this projection_type = 'orthographic is the projection which return 3d globe map'
    fig.update_traces(marker={"opacity": 0.4, 'size': 5, "color": "blue"})
    # layout, exporting html and showing the plot
    fig.update_geos(projection_type="orthographic")
    fig.update_layout(width=800, height=800, margin={
                      "r": 0, "t": 0, "l": 0, "b": 0})
    fig.write_html("3d_plot.html")
    fig.show()

### Plots

In [ ]:
e1_entries = jh.read_items(e1)

In [ ]:
twod_map_coords(e1_entries, samples=20)